In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, GRU, LSTM
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.layers import Layer, Dropout
import cufflinks as cf
cf.go_offline()
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import seaborn as sns

In [3]:
data = pd.read_csv('DailyDelhiClimateTrain.csv', parse_dates=True)
data.head()

,date,meantemp,humidity,wind_speed,meanpressure
0,2013-01-01,10.000000,84.500000,0.000000,1015.666667
1,2013-01-02,7.400000,92.000000,2.980000,1017.800000
2,2013-01-03,7.166667,87.000000,4.633333,1018.666667
3,2013-01-04,8.666667,71.333333,1.233333,1017.166667
4,2013-01-05,6.000000,86.833333,3.700000,1016.500000


In [4]:
data.shape

(1462, 5)

In [3]:
data = data.set_index('date')

In [6]:
data_test =  pd.read_csv('DailyDelhiClimateTest.csv', parse_dates=True)
data_test.head()

,date,meantemp,humidity,wind_speed,meanpressure
0,2017-01-01,15.913043,85.869565,2.743478,59.000000
1,2017-01-02,18.500000,77.222222,2.894444,1018.277778
2,2017-01-03,17.111111,81.888889,4.016667,1018.333333
3,2017-01-04,18.700000,70.050000,4.545000,1015.700000
4,2017-01-05,18.388889,74.944444,3.300000,1014.333333


In [8]:
data_pr = data_test[['meanpressure']]
data_pr.min()

meanpressure    59.0
dtype: float64

In [11]:
data_pr_1 = data[['meanpressure']]
data_pr_1.min()

meanpressure   -3.041667
dtype: float64

In [4]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data[['meantemp']])

In [5]:
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset) - look_back):
        a = dataset[i:(i + look_back), 0]
        X.append(a)
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

look_back = 2  # Например, используем последние 10 значений для прогнозирования следующего
x, y = create_dataset(scaled_data, look_back)

In [6]:
X_dif = x[:-1] - x[1:]
X_new = np.hstack([x[:-1],X_dif])
Y = y[:-1]

X = np.reshape(X_new, (X_new.shape[0], X_new.shape[1], 1))
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, shuffle=False)

In [7]:
class AttentionMechanism(Layer):
    def __init__(self, return_sequences=True):
        super(AttentionMechanism, self).__init__()
        self.return_sequences = return_sequences

    def build(self, input_shape):
        self.W=self.add_weight(name="att_weight", shape=(input_shape[-1],1),
                               initializer="normal")
        self.b=self.add_weight(name="att_bias", shape=(input_shape[1],1),
                               initializer="zeros")

    def call(self, x):
        e = tf.nn.tanh(tf.matmul(x,self.W)+self.b)
        a = tf.nn.softmax(e, axis=1)
        output = x*a

        if self.return_sequences:
            return output

        return tf.reduce_sum(output, axis=1)

In [17]:
import optuna 
from tensorflow.keras.callbacks import EarlyStopping


def create_model(trial):
    model = Sequential()

    n_units_1 = trial.suggest_int('n_units_1', 10, 100)
    n_units_2 = trial.suggest_int('n_units_2', 10, 100)
    n_units_3 = trial.suggest_int('n_units_3', 10, 100)
    
    model.add(GRU(n_units_1, activation='relu', input_shape=(x_train.shape[1], 1), return_sequences=True))
    model.add(GRU(n_units_2, activation='relu', return_sequences=True))
    model.add(Dropout(0.2)),
    model.add(AttentionMechanism(return_sequences=True)),
    model.add(GRU(n_units_3, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    
    return model

def objective(trial):
    # Создание модели
    model = create_model(trial)
    
    # Обучение модели
    early_stopping = EarlyStopping(monitor='val_loss', patience=5)
    history = model.fit(
        x_train, y_train,
        epochs=100,
        batch_size=trial.suggest_int('batch_size', 16, 128),
        validation_data=(x_test, y_test),
        callbacks=[early_stopping],
        verbose=0
    )
    
    # Оценка модели
    Y_pred = model.predict(x_test)
    predicted_power = scaler.inverse_transform(Y_pred)
    y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))
    score = mean_absolute_error(y_test_inv, predicted_power)
    
    return score


study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

# Вывод наилучших гиперпараметров
print("Best hyperparameters:", study.best_params)
print("Best score:", study.best_value)

[I 2024-08-20 11:20:16,719] A new study created in memory with name: no-name-1d9b3c53-4414-4dcf-a9bc-e492efd8fdc8


10/10 [==============================] - 1s 8ms/step


[I 2024-08-20 11:20:48,046] Trial 0 finished with value: 1.1509176955375366 and parameters: {'n_units_1': 85, 'n_units_2': 58, 'n_units_3': 76, 'batch_size': 128}. Best is trial 0 with value: 1.1509176955375366.


10/10 [==============================] - 2s 14ms/step


[I 2024-08-20 11:21:26,672] Trial 1 finished with value: 1.0544993667051044 and parameters: {'n_units_1': 55, 'n_units_2': 12, 'n_units_3': 33, 'batch_size': 94}. Best is trial 1 with value: 1.0544993667051044.


...

10/10 [==============================] - 1s 11ms/step


[I 2024-08-20 11:45:29,074] Trial 18 finished with value: 1.1720408601244787 and parameters: {'n_units_1': 99, 'n_units_2': 72, 'n_units_3': 23, 'batch_size': 66}. Best is trial 14 with value: 0.18270710079007982.


10/10 [==============================] - 1s 10ms/step


[I 2024-08-20 11:48:01,145] Trial 19 finished with value: 0.3968856703867786 and parameters: {'n_units_1': 46, 'n_units_2': 89, 'n_units_3': 62, 'batch_size': 85}. Best is trial 14 with value: 0.18270710079007982.


Best hyperparameters: {'n_units_1': 98, 'n_units_2': 93, 'n_units_3': 50, 'batch_size': 81}
Best score: 0.18270710079007982


In [18]:
model = Sequential()

model.add(GRU(98, activation='relu', input_shape=(x_train.shape[1], 1), return_sequences=True))
model.add(GRU(93, activation='relu', return_sequences=True))
model.add(Dropout(0.2)),
model.add(AttentionMechanism(return_sequences=True)),
model.add(GRU(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_absolute_error')

model.fit(x_train, y_train, epochs=100, batch_size=81, validation_data=(x_test, y_test))

Epoch 1/100
15/15 [==============================] - 4s 68ms/step - loss: 0.4880 - val_loss: 0.4785
Epoch 2/100
15/15 [==============================] - 1s 44ms/step - loss: 0.2291 - val_loss: 0.1052
Epoch 3/100
15/15 [==============================] - 1s 44ms/step - loss: 0.1595 - val_loss: 0.1333
Epoch 4/100
15/15 [==============================] - 1s 44ms/step - loss: 0.1394 - val_loss: 0.1073
Epoch 5/100
15/15 [==============================] - 1s 46ms/step - loss: 0.1127 - val_loss: 0.0661
Epoch 6/100
15/15 [==============================] - 1s 44ms/step - loss: 0.0741 - val_loss: 0.0492
Epoch 7/100
15/15 [==============================] - 1s 47ms/step - loss: 0.0562 - val_loss: 0.0527
Epoch 8/100
15/15 [==============================] - 1s 44ms/step - loss: 0.0558 - val_loss: 0.0469
Epoch 9/100
15/15 [==============================] - 1s 48ms/step - loss: 0.0545 - val_loss: 0.0491
Epoch 10/100
15/15 [==============================] - 1s 81ms/step - loss: 0.0514 - val_loss: 0.0441

In [19]:
predictions = model.predict(x_test)
predicted_power = scaler.inverse_transform(predictions) 

y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))
predictions_inv = scaler.inverse_transform(predictions)
df_test = pd.DataFrame({
    'Y_test': y_test_inv.ravel(),
    'Predictions': predictions_inv.ravel()
})
# Отобразить на графике
df_test.iplot(title="Прогноз vs Реальные значения", xTitle="Время", yTitle="Значение", theme="solar")

10/10 [==============================] - 0s 6ms/step


In [20]:
from sklearn.metrics import mean_absolute_error
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred+0.001) / (y_true+0.001))) * 100
mape = mean_absolute_percentage_error(y_test_inv, predictions_inv)
mae = mean_absolute_error(y_test_inv, predictions_inv)
print(f"MAPE: {mape}%")
print(f"MAE: {mae}")

MAPE: 2.248411817464157%
MAE: 0.6947393462323816


In [13]:
import optuna 
from tensorflow.keras.callbacks import EarlyStopping


def create_model(trial):
    model = Sequential()

    n_units_1 = trial.suggest_int('n_units_1', 10, 100)
    n_units_2 = trial.suggest_int('n_units_2', 10, 100)
    n_units_3 = trial.suggest_int('n_units_3', 10, 100)
    
    model.add(SimpleRNN(n_units_1, activation='relu', input_shape=(x_train.shape[1], 1), return_sequences=True))
    model.add(SimpleRNN(n_units_2, activation='relu', return_sequences=True))
    model.add(Dropout(0.2)),
    model.add(AttentionMechanism(return_sequences=True)),
    model.add(SimpleRNN(n_units_3, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    
    return model

def objective(trial):
    # Создание модели
    model = create_model(trial)
    
    # Обучение модели
    early_stopping = EarlyStopping(monitor='val_loss', patience=5)
    history = model.fit(
        x_train, y_train,
        epochs=100,
        batch_size=trial.suggest_int('batch_size', 16, 128),
        validation_data=(x_test, y_test),
        callbacks=[early_stopping],
        verbose=0
    )
    
    # Оценка модели
    Y_pred = model.predict(x_test)
    predicted_power = scaler.inverse_transform(Y_pred)
    y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))
    score = mean_absolute_error(y_test_inv, predicted_power)
    
    return score


study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

# Вывод наилучших гиперпараметров
print("Best hyperparameters:", study.best_params)
print("Best score:", study.best_value)

[I 2024-08-20 11:06:54,491] A new study created in memory with name: no-name-7e3945d9-74ad-4da0-a224-2cc28c26687f


10/10 [==============================] - 3s 23ms/step


[I 2024-08-20 11:07:07,542] Trial 0 finished with value: 0.8940414714220432 and parameters: {'n_units_1': 83, 'n_units_2': 91, 'n_units_3': 88, 'batch_size': 121}. Best is trial 0 with value: 0.8940414714220432.


10/10 [==============================] - 1s 4ms/step


[I 2024-08-20 11:07:27,653] Trial 1 finished with value: 0.24400145592006145 and parameters: {'n_units_1': 12, 'n_units_2': 63, 'n_units_3': 18, 'batch_size': 55}. Best is trial 1 with value: 0.24400145592006145.


...

10/10 [==============================] - 1s 14ms/step


[I 2024-08-20 11:12:19,301] Trial 18 finished with value: 0.309672516047027 and parameters: {'n_units_1': 23, 'n_units_2': 99, 'n_units_3': 38, 'batch_size': 33}. Best is trial 17 with value: 0.09860592999526686.


10/10 [==============================] - 1s 6ms/step


[I 2024-08-20 11:12:28,594] Trial 19 finished with value: 4.691112352642891 and parameters: {'n_units_1': 22, 'n_units_2': 13, 'n_units_3': 59, 'batch_size': 49}. Best is trial 17 with value: 0.09860592999526686.


Best hyperparameters: {'n_units_1': 29, 'n_units_2': 100, 'n_units_3': 41, 'batch_size': 19}
Best score: 0.09860592999526686


In [14]:
model = Sequential()

model.add(SimpleRNN(29, activation='relu', input_shape=(x_train.shape[1], 1), return_sequences=True))
model.add(SimpleRNN(100, activation='relu', return_sequences=True))
model.add(Dropout(0.2)),
model.add(AttentionMechanism(return_sequences=True)),
model.add(SimpleRNN(41, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_absolute_error')

model.fit(x_train, y_train, epochs=100, batch_size=19, validation_data=(x_test, y_test))

Epoch 1/100
62/62 [==============================] - 11s 70ms/step - loss: 0.1276 - val_loss: 0.0397
Epoch 2/100
62/62 [==============================] - 3s 41ms/step - loss: 0.0463 - val_loss: 0.0393
Epoch 3/100
62/62 [==============================] - 2s 39ms/step - loss: 0.0371 - val_loss: 0.0164
Epoch 4/100
62/62 [==============================] - 2s 35ms/step - loss: 0.0301 - val_loss: 0.0072
Epoch 5/100
62/62 [==============================] - 2s 34ms/step - loss: 0.0252 - val_loss: 0.0303
Epoch 6/100
62/62 [==============================] - 2s 33ms/step - loss: 0.0252 - val_loss: 0.0073
Epoch 7/100
62/62 [==============================] - 2s 33ms/step - loss: 0.0215 - val_loss: 0.0201
Epoch 8/100
62/62 [==============================] - 2s 40ms/step - loss: 0.0201 - val_loss: 0.0053
Epoch 9/100
62/62 [==============================] - 2s 37ms/step - loss: 0.0210 - val_loss: 0.0116
Epoch 10/100
62/62 [==============================] - 3s 48ms/step - loss: 0.0231 - val_loss: 0.031

In [15]:
predictions = model.predict(x_test)
predicted_power = scaler.inverse_transform(predictions) 

y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))
predictions_inv = scaler.inverse_transform(predictions)
df_test = pd.DataFrame({
    'Y_test': y_test_inv.ravel(),
    'Predictions': predictions_inv.ravel()
})
# Отобразить на графике
df_test.iplot(title="Прогноз vs Реальные значения", xTitle="Время", yTitle="Значение", theme="solar")

10/10 [==============================] - 0s 3ms/step


In [16]:
from sklearn.metrics import mean_absolute_error
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred+0.001) / (y_true+0.001))) * 100
mape = mean_absolute_percentage_error(y_test_inv, predictions_inv)
mae = mean_absolute_error(y_test_inv, predictions_inv)
print(f"MAPE: {mape}%")
print(f"MAE: {mae}")

MAPE: 1.9947120802288716%
MAE: 0.6077570048117699


In [ ]:
import optuna 
from tensorflow.keras.callbacks import EarlyStopping


def create_model(trial):
    model = Sequential()

    n_units_1 = trial.suggest_int('n_units_1', 10, 100)
    n_units_2 = trial.suggest_int('n_units_2', 10, 100)
    n_units_3 = trial.suggest_int('n_units_3', 10, 100)
    
    model.add(LSTM(n_units_1, activation='relu', input_shape=(x_train.shape[1], 1), return_sequences=True))
    model.add(LSTM(n_units_2, activation='relu', return_sequences=True))
    model.add(Dropout(0.2)),
    model.add(AttentionMechanism(return_sequences=True)),
    model.add(LSTM(n_units_3, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    
    return model

def objective(trial):
    # Создание модели
    model = create_model(trial)
    
    # Обучение модели
    early_stopping = EarlyStopping(monitor='val_loss', patience=5)
    history = model.fit(
        x_train, y_train,
        epochs=100,
        batch_size=trial.suggest_int('batch_size', 16, 128),
        validation_data=(x_test, y_test),
        callbacks=[early_stopping],
        verbose=0
    )
    
    # Оценка модели
    Y_pred = model.predict(x_test)
    predicted_power = scaler.inverse_transform(Y_pred)
    y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))
    score = mean_absolute_error(y_test_inv, predicted_power)
    
    return score


study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

# Вывод наилучших гиперпараметров
print("Best hyperparameters:", study.best_params)
print("Best score:", study.best_value)

In [22]:
model = Sequential()

model.add(LSTM(98, activation='relu', input_shape=(x_train.shape[1], 1), return_sequences=True))
model.add(LSTM(79, activation='relu', return_sequences=True))
model.add(Dropout(0.2)),
model.add(AttentionMechanism(return_sequences=True)),
model.add(LSTM(44, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_absolute_error')

model.fit(x_train, y_train, epochs=100, batch_size=47, validation_data=(x_test, y_test))

Epoch 1/100
25/25 [==============================] - 9s 164ms/step - loss: 0.5106 - val_loss: 0.5513
Epoch 2/100
25/25 [==============================] - 5s 192ms/step - loss: 0.2321 - val_loss: 0.1745
Epoch 3/100
25/25 [==============================] - 6s 229ms/step - loss: 0.1314 - val_loss: 0.0806
Epoch 4/100
25/25 [==============================] - 5s 196ms/step - loss: 0.0930 - val_loss: 0.0619
Epoch 5/100
25/25 [==============================] - 6s 227ms/step - loss: 0.0810 - val_loss: 0.1030
Epoch 6/100
25/25 [==============================] - 5s 214ms/step - loss: 0.0759 - val_loss: 0.0651
Epoch 7/100
25/25 [==============================] - 6s 223ms/step - loss: 0.0709 - val_loss: 0.0621
Epoch 8/100
25/25 [==============================] - 7s 266ms/step - loss: 0.0694 - val_loss: 0.0591
Epoch 9/100
25/25 [==============================] - 5s 194ms/step - loss: 0.0675 - val_loss: 0.0592
Epoch 10/100
25/25 [==============================] - 3s 137ms/step - loss: 0.0654 - val_lo

In [24]:
predictions = model.predict(x_test)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))
predictions_inv = scaler.inverse_transform(predictions)
df_test = pd.DataFrame({
    'Y_test': y_test_inv.ravel(),
    'Predictions': predictions_inv.ravel()
})
# Отобразить на графике
df_test.iplot(title="Прогноз vs Реальные значения", xTitle="Время", yTitle="Значение", theme="solar")

10/10 [==============================] - 0s 4ms/step


In [25]:
from sklearn.metrics import mean_absolute_error
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred+0.001) / (y_true+0.001))) * 100
mape = mean_absolute_percentage_error(y_test_inv, predictions_inv)
mae = mean_absolute_error(y_test_inv, predictions_inv)
print(f"MAPE: {mape}%")
print(f"MAE: {mae}")

MAPE: 5.222672811452879%
MAE: 1.6227180249193076


In [26]:
df = data[['meantemp', 'humidity', 'wind_speed']]
df.head()

,meantemp,humidity,wind_speed
date,,,
2013-01-01,10.000000,84.500000,0.000000
2013-01-02,7.400000,92.000000,2.980000
2013-01-03,7.166667,87.000000,4.633333
2013-01-04,8.666667,71.333333,1.233333
2013-01-05,6.000000,86.833333,3.700000


In [27]:
train_size = int(len(df) * 0.8)
dl_train, dl_test = df.iloc[:train_size], df.iloc[train_size:]
print(len(dl_train), len(dl_test))

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data[['meantemp']])



1169 293


In [21]:
from sklearn.preprocessing import RobustScaler, MinMaxScaler

robust_scaler = RobustScaler() 
minmax_scaler = MinMaxScaler()  
target_transformer = MinMaxScaler() 

In [22]:
dl_train['wind_speed'] = robust_scaler.fit_transform(dl_train[['wind_speed']])
dl_train['humidity'] = minmax_scaler.fit_transform(dl_train[['humidity']])
dl_train['meantemp'] = target_transformer.fit_transform(dl_train[['meantemp']])

dl_test['wind_speed'] = robust_scaler.transform(dl_test[['wind_speed']])
dl_test['humidity'] = minmax_scaler.transform(dl_test[['humidity']])
dl_test['meantemp'] = target_transformer.transform(dl_test[['meantemp']])

In [40]:
def create_dataset(X, y, time_steps=1):  
    Xs, ys = [], []   
    for i in range(len(X) - time_steps):   
        v = X.iloc[i:(i + time_steps)].values 
        Xs.append(v)      
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)  

In [41]:
look_back = 10 
X_train, y_train = create_dataset(dl_train, dl_train['meantemp'], look_back)
X_test, y_test = create_dataset(dl_test, dl_test['meantemp'], look_back)


X_dif = x[:-1] - x[1:]
X_new = np.hstack([x[:-1],X_dif])
Y = y[:-1]

X = np.reshape(X_new, (X_new.shape[0], X_new.shape[1], 1))
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, shuffle=False)

In [42]:
rnn_model = Sequential()
rnn_model.add(SimpleRNN(100, activation='relu', input_shape=(look_back, X_train.shape[2])))
rnn_model.add(Dense(1))
rnn_model.compile(optimizer='adam', loss='mae')

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = rnn_model.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test), batch_size=1, callbacks=[early_stopping])

loss = rnn_model.evaluate(X_test, y_test)
print(f'Validation Loss: {loss}')

Epoch 1/30
1159/1159 [==============================] - 3s 2ms/step - loss: 0.0801 - val_loss: 0.0542
Epoch 2/30
1159/1159 [==============================] - 2s 2ms/step - loss: 0.0550 - val_loss: 0.0538
Epoch 3/30
1159/1159 [==============================] - 2s 2ms/step - loss: 0.0518 - val_loss: 0.0531
Epoch 4/30
1159/1159 [==============================] - 2s 2ms/step - loss: 0.0470 - val_loss: 0.0606
Epoch 5/30
1159/1159 [==============================] - 2s 2ms/step - loss: 0.0461 - val_loss: 0.0413
Epoch 6/30
1159/1159 [==============================] - 3s 2ms/step - loss: 0.0443 - val_loss: 0.0400
Epoch 7/30
1159/1159 [==============================] - 2s 2ms/step - loss: 0.0432 - val_loss: 0.0424
Epoch 8/30
1159/1159 [==============================] - 2s 2ms/step - loss: 0.0425 - val_loss: 0.0428
Epoch 9/30
1159/1159 [==============================] - 3s 3ms/step - loss: 0.0419 - val_loss: 0.0463
Epoch 10/30
1159/1159 [==============================] - 4s 3ms/step - loss: 0.042

In [43]:
rnn_pred = rnn_model.predict(X_test)
rnn_pred = target_transformer.inverse_transform(rnn_pred)

y_test = y_test.reshape(-1, 1)
y_test = target_transformer.inverse_transform(y_test)

9/9 [==============================] - 0s 2ms/step


In [56]:
rnn_mape = mean_absolute_percentage_error(y_test, rnn_pred)
rnn_mae = mean_absolute_error(y_test, rnn_pred)
print(f"MAPE: {rnn_mape} %")
print(f"MAE: {rnn_mae}")

MAPE: 4.641916970646035 %
MAE: 1.2542197165289939


In [45]:
df_test = pd.DataFrame({
    'Y_test': y_test.ravel(),
    'Predictions': rnn_pred.ravel()
})
# Отобразить на графике
df_test.iplot(title="Прогноз vs Реальные значения", xTitle="Время", yTitle="Значение", theme="solar")

In [47]:
look_back = 10 
X_train, y_train = create_dataset(dl_train, dl_train['meantemp'], look_back)
X_test, y_test = create_dataset(dl_test, dl_test['meantemp'], look_back)

In [48]:
lstm_model = Sequential()
lstm_model.add(LSTM(100, activation='relu', input_shape=(look_back, X_train.shape[2])))
lstm_model.add(Dense(1))
lstm_model.compile(optimizer='adam', loss='mae')

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = lstm_model.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test), batch_size=1, callbacks=[early_stopping])


loss = lstm_model.evaluate(X_test, y_test)
print(f'Validation Loss: {loss}')

Epoch 1/30
1159/1159 [==============================] - 4s 3ms/step - loss: 0.0807 - val_loss: 0.0543
Epoch 2/30
1159/1159 [==============================] - 24s 21ms/step - loss: 0.0530 - val_loss: 0.0496
Epoch 3/30
1159/1159 [==============================] - 19s 16ms/step - loss: 0.0503 - val_loss: 0.0446
Epoch 4/30
1159/1159 [==============================] - 12s 10ms/step - loss: 0.0482 - val_loss: 0.0438
Epoch 5/30
1159/1159 [==============================] - 12s 10ms/step - loss: 0.0452 - val_loss: 0.0398
Epoch 6/30
1159/1159 [==============================] - 16s 13ms/step - loss: 0.0442 - val_loss: 0.0443
Epoch 7/30
1159/1159 [==============================] - 16s 14ms/step - loss: 0.0432 - val_loss: 0.0434
Epoch 8/30
1159/1159 [==============================] - 17s 15ms/step - loss: 0.0416 - val_loss: 0.0391
Epoch 9/30
1159/1159 [==============================] - 14s 12ms/step - loss: 0.0423 - val_loss: 0.0407
Epoch 10/30
1159/1159 [==============================] - 16s 13ms/

In [49]:
lstm_pred = lstm_model.predict(X_test)
lstm_pred = target_transformer.inverse_transform(lstm_pred)

y_test = y_test.reshape(-1, 1)
y_test = target_transformer.inverse_transform(y_test)

9/9 [==============================] - 0s 2ms/step


In [57]:
lstm_mape = mean_absolute_percentage_error(y_test, lstm_pred)
lstm_mae = mean_absolute_error(y_test, lstm_pred)
print(f"MAPE: {lstm_mape} %")
print(f"MAE: {lstm_mae}")

df_test = pd.DataFrame({
    'Y_test': y_test.ravel(),
    'Predictions': lstm_pred.ravel()
})
# Отобразить на графике
df_test.iplot(title="Прогноз vs Реальные значения", xTitle="Время", yTitle="Значение", theme="solar")


MAPE: 4.783140829793508 %
MAE: 1.2799382625371212


In [60]:
look_back = 10 
X_train, y_train = create_dataset(dl_train, dl_train['meantemp'], look_back)
X_test, y_test = create_dataset(dl_test, dl_test['meantemp'], look_back)

In [61]:
gru_model = Sequential()
gru_model.add(GRU(100, activation='relu', input_shape=(look_back, X_train.shape[2])))
gru_model.add(Dense(1))
gru_model.compile(optimizer='adam', loss='mae')

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = gru_model.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test), batch_size=1, callbacks=[early_stopping])


loss = gru_model.evaluate(X_test, y_test)
print(f'Validation Loss: {loss}')

Epoch 1/30
1159/1159 [==============================] - 4s 3ms/step - loss: 0.0725 - val_loss: 0.0483
Epoch 2/30
1159/1159 [==============================] - 7s 6ms/step - loss: 0.0483 - val_loss: 0.0449
Epoch 3/30
1159/1159 [==============================] - 14s 12ms/step - loss: 0.0469 - val_loss: 0.0411
Epoch 4/30
1159/1159 [==============================] - 13s 12ms/step - loss: 0.0441 - val_loss: 0.0449
Epoch 5/30
1159/1159 [==============================] - 11s 9ms/step - loss: 0.0436 - val_loss: 0.0627
Epoch 6/30
1159/1159 [==============================] - 11s 9ms/step - loss: 0.0426 - val_loss: 0.0459
Epoch 7/30
1159/1159 [==============================] - 11s 9ms/step - loss: 0.0418 - val_loss: 0.0414
Epoch 8/30
1159/1159 [==============================] - 10s 9ms/step - loss: 0.0413 - val_loss: 0.0389
Epoch 9/30
1159/1159 [==============================] - 10s 9ms/step - loss: 0.0404 - val_loss: 0.0430
Epoch 10/30
1159/1159 [==============================] - 11s 9ms/step - l

In [62]:
gru_pred = gru_model.predict(X_test)
gru_pred = target_transformer.inverse_transform(gru_pred)

y_test = y_test.reshape(-1, 1)
y_test = target_transformer.inverse_transform(y_test)


gru_mape = mean_absolute_percentage_error(y_test, gru_pred)
gru_mae = mean_absolute_error(y_test, gru_pred)
print(f"MAPE: {gru_mape} %")
print(f"MAE: {gru_mae}")

df_test = pd.DataFrame({
    'Y_test': y_test.ravel(),
    'Predictions': gru_pred.ravel()
})
# Отобразить на графике
df_test.iplot(title="Прогноз vs Реальные значения", xTitle="Время", yTitle="Значение", theme="solar")

9/9 [==============================] - 0s 5ms/step
MAPE: 4.584657950797873 %
MAE: 1.2369458593343154


In [63]:
dl_compare_without_mech = {
        'SimpleRNN': [rnn_mape, rnn_mae], 
        'LSTM': [lstm_mape, lstm_mae],
        'GRU ': [gru_mape,gru_mae]
            }
dl_compare_without_mech = pd.DataFrame(dl_compare_without_mech,index=['MAPE','MAE'])
dl_compare_without_mech.head()

,SimpleRNN,LSTM,GRU
MAPE,4.641917,4.783141,4.584658
MAE,1.254220,1.279938,1.236946


## Вариант с исходной моделью и механизмом внимания

In [64]:
look_back = 10 
X_train, y_train = create_dataset(dl_train, dl_train['meantemp'], look_back)
X_test, y_test = create_dataset(dl_test, dl_test['meantemp'], look_back)

gru_model_a = Sequential()

gru_model_a.add(GRU(94, activation='relu', input_shape=(look_back, X_train.shape[2]), return_sequences=True))
gru_model_a.add(GRU(100, activation='relu', return_sequences=True))
gru_model_a.add(Dropout(0.2)),
gru_model_a.add(AttentionMechanism(return_sequences=True)),
gru_model_a.add(GRU(14, activation='relu'))
gru_model_a.add(Dense(1))
gru_model_a.compile(optimizer='adam', loss='mean_absolute_error')

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = gru_model_a.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), batch_size=10, callbacks=[early_stopping])

loss = gru_model_a.evaluate(X_test, y_test)
print(f'Validation Loss: {loss}')

Epoch 1/100
116/116 [==============================] - 14s 79ms/step - loss: 0.2004 - val_loss: 0.0798
Epoch 2/100
116/116 [==============================] - 10s 84ms/step - loss: 0.0704 - val_loss: 0.0598
Epoch 3/100
116/116 [==============================] - 9s 81ms/step - loss: 0.0645 - val_loss: 0.0758
Epoch 4/100
116/116 [==============================] - 7s 60ms/step - loss: 0.0659 - val_loss: 0.0593
Epoch 5/100
116/116 [==============================] - 8s 70ms/step - loss: 0.0606 - val_loss: 0.0603
Epoch 6/100
116/116 [==============================] - 8s 69ms/step - loss: 0.0585 - val_loss: 0.0581
Epoch 7/100
116/116 [==============================] - 8s 68ms/step - loss: 0.0567 - val_loss: 0.0606
Epoch 8/100
116/116 [==============================] - 7s 64ms/step - loss: 0.0558 - val_loss: 0.0596
Epoch 9/100
116/116 [==============================] - 8s 73ms/step - loss: 0.0583 - val_loss: 0.0545
Epoch 10/100
116/116 [==============================] - 7s 63ms/step - loss: 0.0

In [65]:
gru_pred_a = gru_model_a.predict(X_test)
gru_pred_a = target_transformer.inverse_transform(gru_pred_a)

y_test = y_test.reshape(-1, 1)
y_test = target_transformer.inverse_transform(y_test)


gru_mape_a = mean_absolute_percentage_error(y_test, gru_pred_a)
gru_mae_a = mean_absolute_error(y_test, gru_pred_a)
print(f"MAPE: {gru_mape_a} %")
print(f"MAE: {gru_mae_a}")

df_test = pd.DataFrame({
    'Y_test': y_test.ravel(),
    'Predictions': gru_pred_a.ravel()
})
# Отобразить на графике
df_test.iplot(title="Прогноз vs Реальные значения", xTitle="Время", yTitle="Значение", theme="solar")

9/9 [==============================] - 0s 9ms/step
MAPE: 4.686335126252312 %
MAE: 1.281806391977061


In [67]:
look_back = 10 
X_train, y_train = create_dataset(dl_train, dl_train['meantemp'], look_back)
X_test, y_test = create_dataset(dl_test, dl_test['meantemp'], look_back)

rnn_model_a = Sequential()

rnn_model_a.add(SimpleRNN(62, activation='relu', input_shape=(look_back, X_train.shape[2]), return_sequences=True))
rnn_model_a.add(SimpleRNN(29, activation='relu', return_sequences=True))
rnn_model_a.add(Dropout(0.2)),
rnn_model_a.add(AttentionMechanism(return_sequences=True)),
rnn_model_a.add(SimpleRNN(15, activation='relu'))
rnn_model_a.add(Dense(1))
rnn_model_a.compile(optimizer='adam', loss='mean_absolute_error')

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = rnn_model_a.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), batch_size=47, callbacks=[early_stopping])

loss = rnn_model_a.evaluate(X_test, y_test)
print(f'Validation Loss: {loss}')

Epoch 1/100
25/25 [==============================] - 5s 78ms/step - loss: 0.3584 - val_loss: 0.1246
Epoch 2/100
25/25 [==============================] - 1s 30ms/step - loss: 0.1278 - val_loss: 0.0738
Epoch 3/100
25/25 [==============================] - 1s 29ms/step - loss: 0.0821 - val_loss: 0.0665
Epoch 4/100
25/25 [==============================] - 1s 30ms/step - loss: 0.0719 - val_loss: 0.0640
Epoch 5/100
25/25 [==============================] - 1s 39ms/step - loss: 0.0685 - val_loss: 0.0635
Epoch 6/100
25/25 [==============================] - 1s 56ms/step - loss: 0.0648 - val_loss: 0.0637
Epoch 7/100
25/25 [==============================] - 1s 52ms/step - loss: 0.0616 - val_loss: 0.0676
Epoch 8/100
25/25 [==============================] - 1s 48ms/step - loss: 0.0581 - val_loss: 0.0597
Epoch 9/100
25/25 [==============================] - 1s 59ms/step - loss: 0.0588 - val_loss: 0.0573
Epoch 10/100
25/25 [==============================] - 1s 55ms/step - loss: 0.0593 - val_loss: 0.0663

In [68]:
rnn_pred_a = rnn_model_a.predict(X_test)
rnn_pred_a = target_transformer.inverse_transform(rnn_pred_a)

y_test = y_test.reshape(-1, 1)
y_test = target_transformer.inverse_transform(y_test)


rnn_mape_a = mean_absolute_percentage_error(y_test, rnn_pred_a)
rnn_mae_a = mean_absolute_error(y_test, rnn_pred_a)
print(f"MAPE: {rnn_mape_a} %")
print(f"MAE: {rnn_mae_a}")

df_test = pd.DataFrame({
    'Y_test': y_test.ravel(),
    'Predictions': rnn_pred_a.ravel()
})
# Отобразить на графике
df_test.iplot(title="Прогноз vs Реальные значения", xTitle="Время", yTitle="Значение", theme="solar")

9/9 [==============================] - 0s 3ms/step
MAPE: 5.957408406289591 %
MAE: 1.6212015979999124


In [69]:
look_back = 10 
X_train, y_train = create_dataset(dl_train, dl_train['meantemp'], look_back)
X_test, y_test = create_dataset(dl_test, dl_test['meantemp'], look_back)

lstm_model_a = Sequential()

lstm_model_a.add(LSTM(90, activation='relu', input_shape=(look_back, X_train.shape[2]), return_sequences=True))
lstm_model_a.add(LSTM(79, activation='relu', return_sequences=True))
lstm_model_a.add(Dropout(0.2)),
lstm_model_a.add(AttentionMechanism(return_sequences=True)),
lstm_model_a.add(LSTM(30, activation='relu'))
lstm_model_a.add(Dense(1))
lstm_model_a.compile(optimizer='adam', loss='mean_absolute_error')

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = lstm_model_a.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), batch_size=35, callbacks=[early_stopping])

loss = lstm_model_a.evaluate(X_test, y_test)
print(f'Validation Loss: {loss}') 

Epoch 1/100
34/34 [==============================] - 20s 242ms/step - loss: 0.4524 - val_loss: 0.1895
Epoch 2/100
34/34 [==============================] - 5s 141ms/step - loss: 0.1179 - val_loss: 0.0639
Epoch 3/100
34/34 [==============================] - 5s 161ms/step - loss: 0.0799 - val_loss: 0.0719
Epoch 4/100
34/34 [==============================] - 5s 156ms/step - loss: 0.0714 - val_loss: 0.0597
Epoch 5/100
34/34 [==============================] - 6s 172ms/step - loss: 0.0646 - val_loss: 0.0627
Epoch 6/100
34/34 [==============================] - 6s 177ms/step - loss: 0.0644 - val_loss: 0.0651
Epoch 7/100
34/34 [==============================] - 5s 149ms/step - loss: 0.0640 - val_loss: 0.0600
Epoch 8/100
34/34 [==============================] - 5s 156ms/step - loss: 0.0631 - val_loss: 0.0622
Epoch 9/100
34/34 [==============================] - 5s 137ms/step - loss: 0.0632 - val_loss: 0.0586
Epoch 10/100
34/34 [==============================] - 5s 139ms/step - loss: 0.0643 - val_l

In [70]:
lstm_pred_a = lstm_model_a.predict(X_test)
lstm_pred_a = target_transformer.inverse_transform(lstm_pred_a)

y_test = y_test.reshape(-1, 1)
y_test = target_transformer.inverse_transform(y_test)


lstm_mape_a = mean_absolute_percentage_error(y_test, lstm_pred_a)
lstm_mae_a = mean_absolute_error(y_test, lstm_pred_a)
print(f"MAPE: {lstm_mape_a} %")
print(f"MAE: {lstm_mae_a}")

df_test = pd.DataFrame({
    'Y_test': y_test.ravel(),
    'Predictions': lstm_pred_a.ravel()
})
# Отобразить на графике
df_test.iplot(title="Прогноз vs Реальные значения", xTitle="Время", yTitle="Значение", theme="solar")

9/9 [==============================] - 0s 9ms/step
MAPE: 6.805334727647154 %
MAE: 1.8831524487066187


In [71]:
dl_compare_with_mech = {
        'SimpleRNN_A': [rnn_mape_a, rnn_mae_a], 
        'LSTM_A': [lstm_mape_a, lstm_mae_a],
        'GRU_A': [gru_mape_a, gru_mae_a]
            }
dl_compare_with_mech = pd.DataFrame(dl_compare_with_mech,index=['MAPE','MAE'])
dl_compare_with_mech.head()

,SimpleRNN_A,LSTM_A,GRU_A
MAPE,5.957408,6.805335,4.686335
MAE,1.621202,1.883152,1.281806


In [72]:
compare_models_df = pd.concat([dl_compare_without_mech, dl_compare_with_mech], axis=1)
compare_models_df.head()

,SimpleRNN,LSTM,GRU,SimpleRNN_A,LSTM_A,GRU_A
MAPE,4.641917,4.783141,4.584658,5.957408,6.805335,4.686335
MAE,1.254220,1.279938,1.236946,1.621202,1.883152,1.281806


### При наличии трех признаков (wind_speed, humidity, meantemp) и таргета - meantemp лучше всего себя показала модель GRU без использования механизма внимания. Выводы основаны на ошибке MAE, которая равна 1.236946.